In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# 1. 데이터 로드
# pandas.read_excel()을 사용해 엑셀 파일에서 교통량 데이터를 불러옵니다. (참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html)
df = pd.read_excel('../data/weekday_traffic.xlsx')
df.head()

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,17시,18시,19시,20시,21시,22시,23시,혼잡,평균교통량,새로운 혼잡
0,20240801,목,마포대교,C-07,유입,마포대교남단->마포대교북단,1121.100127,739.396817,603.646236,429.588089,...,3982.454958,3465.886595,2569.941716,2727.605192,2393.113295,2159.360088,1520.158999,True,2564.041667,False
1,20240802,금,마포대교,C-07,유입,마포대교남단->마포대교북단,1064.668712,758.031761,487.068186,397.855528,...,3890.244416,3425.769366,2954.749926,2613.879546,2301.021973,2244.874364,1496.910955,True,2516.208333,False
2,20240805,월,마포대교,C-07,유입,마포대교남단->마포대교북단,616.797360,464.364338,337.000433,282.774209,...,3865.243889,3743.189576,2772.238162,2506.984733,2199.044578,2080.754404,1571.072355,False,2355.000000,False
3,20240806,화,마포대교,C-07,유입,마포대교남단->마포대교북단,1063.667580,687.378212,416.206495,339.725827,...,3839.879003,4042.135076,2663.051395,2346.262683,2647.551173,1916.299569,1523.996405,False,2488.666667,False
4,20240807,수,마포대교,C-07,유입,마포대교남단->마포대교북단,1042.007691,754.815160,531.468487,363.953378,...,3840.413752,3650.694194,2892.151038,2472.565715,2729.293114,2335.362316,1478.987119,True,2549.875000,False


In [3]:
# 2. 독립 변수(X)와 종속 변수(y) 설정
# 시간대별 교통량 데이터를 독립 변수(X)로, 혼잡 여부 데이터를 종속 변수(y)로 설정합니다.
X = df.loc[:, '0시':'23시']  # 시간대별 교통량 데이터를 독립 변수로 설정
y = df['혼잡']  # 혼잡 여부를 종속 변수로 설정

In [4]:
# 3. 학습 데이터와 테스트 데이터 분리
# train_test_split()을 사용해 데이터를 훈련 세트(70%)와 테스트 세트(30%)로 분리합니다.
# stratify=y 옵션은 종속 변수(y)의 클래스 비율을 유지하도록 합니다. (참고: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [5]:
# 4. 데이터 스케일링 (표준화)
# StandardScaler()를 사용해 데이터의 평균을 0, 표준편차를 1로 맞추어 표준화합니다.
# Standardization은 피처들의 분포 모양(평균과 분산)이 서로 다를 때, 이를 평균 0, 표준편차 1로 맞춰서 모델이 특성마다 동일한 기준으로 학습할 수 있게 도와줍니다.

# 훈련 데이터에서 스케일링 기준을 학습하고, 학습된 기준으로 훈련 및 테스트 데이터를 변환합니다. (참고: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
scaler = StandardScaler()
# scaler를 사용해 변환합니다.
# fit_transform(): 훈련 데이터에 대해 평균과 표준편차를 계산하고, 그 값으로 데이터를 표준화합니다.
# transform(): 훈련 데이터에서 계산된 평균과 표준편차를 사용하여 테스트 데이터를 표준화합니다.
X_train_scaled = scaler.fit_transform(X_train)  # 훈련 데이터를 스케일링
X_test_scaled = scaler.transform(X_test)  # 테스트 데이터를 스케일링

In [6]:
# 5. KNN 분류 모델 생성 및 학습
# KNeighborsClassifier()를 사용하여 KNN 분류 모델을 생성하고, 훈련 데이터를 이용해 학습시킵니다.
# n_neighbors=5로 설정해, K값을 5로 사용하여 가장 가까운 5개의 이웃을 기준으로 예측합니다. (참고: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [ ]:
# 6. 테스트 셋에서 첫 번째 샘플 선택 및 결과 출력
# 무작위 선택 대신, 첫 번째 샘플을 선택하여 항상 일관된 결과를 보장합니다.
selected_sample = X_test.iloc[[0]]  # 테스트 세트에서 첫 번째 샘플을 선택
selected_sample_scaled = X_test_scaled[0].reshape(1, -1)  # 선택한 샘플을 스케일링된 값으로 변환

In [8]:
# 7. 해당 샘플의 실제 혼잡 여부
# y_test에서 첫 번째 샘플의 실제 혼잡 여부를 가져옵니다.
actual_congestion = y_test.iloc[0]

In [9]:
# 8. 모델을 사용해 예측된 혼잡 여부
# KNN 모델을 사용해 첫 번째 샘플의 혼잡 여부를 예측합니다. (참고: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
predicted_congestion = knn.predict(selected_sample_scaled)

In [10]:
# 9. 해당 샘플의 날짜 추출
# df에서 선택된 샘플의 날짜 정보를 가져옵니다.
selected_date = df.iloc[X_test.index[0]]['일자']

In [11]:
# 10. 결과 출력
# 첫 번째 샘플의 실제 혼잡 여부와 예측된 혼잡 여부를 비교하여 출력합니다.
print(f"선택된 날짜: {selected_date}")
print(f"실제 혼잡 여부: {'혼잡' if actual_congestion else '비혼잡'}")
print(f"예측 혼잡 여부: {'혼잡' if predicted_congestion else '비혼잡'}")

선택된 날짜: 20240809
실제 혼잡 여부: 비혼잡
예측 혼잡 여부: 비혼잡
